# ch10 데이터베이스 연동

## SQLite 데이터 베이스 연결

### SQLite & python
- SQLite 데이터 베이스는 별도의 서버 프로세스 없이 SQL을 사용하여 엑세스할 수 있도록 만든 간단한 디스크 기반 데이터베이스를 제공하는 c라이브러리
- SQLite는 프로토타입을 만들 때까지 사용
- DB browser for SQLite

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.33.0'

### 데이터베이스 연결

In [91]:
# DB 연결 객체 생성.
conn = sqlite3.connect('data/ch10_example.db')

In [92]:
# 커서 객체: SQL문을 실행시키고 결과 데이터를 조회하는 데 사용
cursor = conn.cursor()

In [93]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
""")

OperationalError: table MEMBER already exists

In [10]:
cursor.execute("""
    DROP TABLE MEMBER
""")

In [23]:
cursor.execute("INSERT INTO MEMBER VALUES ('HONG', 20, 'WWW@W.COM')")
cursor.execute("INSERT INTO MEMBER VALUES ('JUNG', 23, 'QQQ@W.COM')")
cursor.execute("INSERT INTO MEMBER VALUES ('LEE', 26, 'EEE@W.COM')")
conn.commit()

In [30]:
cursor.execute("SELECT * FROM MEMBER") #결과는 cursor가 가르킴
#결과 출력: fetchall(), fetchone(), fetchmany(n)
members = cursor.fetchall()

In [17]:
for member in members:
    print(member)

('HONG', 20, 'WWW@W.COM')
('JUNG', 23, 'QQQ@W.COM')
('LEE', 26, 'EEE@W.COM')
('HONG', 20, 'WWW@W.COM')
('JUNG', 23, 'QQQ@W.COM')
('LEE', 26, 'EEE@W.COM')


In [31]:
cursor.execute("SELECT * FROM MEMBER")
while cursor.fetchone() != None:
    print(cursor.fetchone())

('JUNG', 23, 'QQQ@W.COM')
('HONG', 20, 'WWW@W.COM')
('LEE', 26, 'EEE@W.COM')
('JUNG', 23, 'QQQ@W.COM')
('HONG', 20, 'WWW@W.COM')
('LEE', 26, 'EEE@W.COM')
('JUNG', 23, 'QQQ@W.COM')
None


In [24]:
cursor.execute("SELECT * FROM MEMBER")
cursor.fetchmany(2)

[('HONG', 20, 'WWW@W.COM'), ('JUNG', 23, 'QQQ@W.COM')]

In [32]:
cursor.close()
conn.close()

### SQL 구문에 파라미터 사용하기
- qmark, named

In [33]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE EMAIL IN ('WWW@W.COM', 'EEE@W.COM')")
cursor.fetchall()

[('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM')]

In [40]:
#qmark
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", ('HONG', 'LEE'))
cursor.fetchall()

[('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM')]

In [41]:
name1 = input('이름1: ')
name2 = input('이름2: ')
names = (name1, name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
cursor.fetchall()

이름1: HONG
이름2: LEE


[('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM'),
 ('HONG', 20, 'WWW@W.COM'),
 ('LEE', 26, 'EEE@W.COM')]

In [42]:
name1 = input('이름1: ')
name2 = input('이름2: ')
names = [name1, name2]
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
cursor.fetchall()

이름1: 
이름2: 


[]

In [47]:
# named
name = input("검색하고자 하는 이름: ")
cursor.execute("SELECT * FROM MEMBER WHERE NAME=:name", {'name':name}) #딕셔너리로 입력
if len(members)==0:
    print('empty')
else:
    print(cursor.fetchall())

검색하고자 하는 이름: HONG
[('HONG', 20, 'WWW@W.COM'), ('HONG', 20, 'WWW@W.COM'), ('HONG', 20, 'WWW@W.COM'), ('HONG', 20, 'WWW@W.COM'), ('HONG', 20, 'WWW@W.COM')]


In [48]:
cursor.close()
conn.close()

## Oracle 데이터 베이스 연결

In [95]:
#데이터 베이스 설정(DBMS가 있는 서버 주소, 포트 번호, sid)
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [96]:
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
conn#연결

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [97]:
conn.close()

In [98]:
#conn 객체 만드는 방법 1
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid="xe")
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
conn.close()

In [99]:
#conn 객체 만드는 방법 2
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [100]:
#cursor 얻어오기
cursor = conn.cursor()

In [101]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
m = cursor.fetchall()
type(m)

list

In [102]:
sql = "SELECT * FROM EMP WHERE DEPTNO = :deptno"
deptno = input("원하는 부서번호: ")
cursor.execute(sql, {"deptno": deptno})
for e in cursor:
    print(e)

원하는 부서번호: 


In [103]:
import pandas as pd
cursor.execute("SELECT * FROM EMP")
data = cursor.fetchall() # 튜플 리스트
print(data)
data_df = pd.DataFrame(data) # 데이터 프레임 타입
data_df

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [67]:
#각 필드 특징 정보
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [70]:
data_df.columns = [row[0] for row in cursor.description]

In [71]:
data_df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [72]:
cursor.close()
conn.close()

## MariaDB 데이터 베이스 연결

In [74]:
import pymysql

In [76]:
conn = pymysql.connect(host="localhost", port=3306, db="kimdb", user="root", 
                       passwd="mysql", charset="utf8", autocommit=True)

In [79]:
cursor = conn.cursor()
sql = "select * from personal"
cursor.execute(sql)
result = cursor.fetchall()
for e in result:
    print(e)

(1001, 'bill', 'president', None, datetime.date(1989, 1, 10), 7000, None, 10)
(1111, 'smith', 'manager', 1001, datetime.date(1990, 12, 17), 1000, None, 10)
(1112, 'ally', 'salesman', 1116, datetime.date(1991, 2, 20), 1600, 500, 30)
(1113, 'word', 'salesman', 1116, datetime.date(1992, 2, 24), 1450, 300, 30)
(1114, 'james', 'manager', 1001, datetime.date(1990, 4, 12), 3975, None, 20)
(1116, 'johnson', 'manager', 1001, datetime.date(1991, 5, 1), 3550, None, 30)
(1118, 'martin', 'analyst', 1111, datetime.date(1991, 9, 9), 3450, None, 10)
(1121, 'kim', 'clerk', 1114, datetime.date(1990, 12, 8), 4000, None, 20)
(1123, 'lee', 'salesman', 1116, datetime.date(1991, 9, 23), 1200, 0, 30)
(1226, 'park', 'analyst', 1111, datetime.date(1990, 1, 3), 2500, None, 10)


In [80]:
personal = pd.DataFrame(result)
personal.columns = [row[0] for row in cursor.description]
personal.head()

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20


In [81]:
cursor.close()
conn.close()